## Train dataset on MNIST :(

In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
from torch.utils.data import DataLoader

import torch.optim as optim
import torch.nn as nn

import torchvision.transforms as transforms

import numpy as np

import torchvision
import torch

In [2]:
from tqdm import tqdm

In [3]:
train_transform = transforms.Compose(
    [transforms.ToTensor()])

val_transforms = transforms.Compose(
    [transforms.ToTensor()])

train_set = torchvision.datasets.MNIST("./", train=True, transform=train_transform, target_transform=None, download=True)
val_set = torchvision.datasets.MNIST("./", train=False, transform=val_transforms, target_transform=None, download=True)

train_loader = DataLoader(train_set, batch_size=256)
val_loader = DataLoader(val_set, batch_size=256)

In [3]:
device = 'cuda'

In [5]:
# net = torchvision.models.resnet18(pretrained=True)
net = torchvision.models.resnet50(pretrained=True)

# Replace 1st layer to use it on grayscale images
net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
net.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
net = net.to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [7]:
n_epochs = 8

In [8]:
def train():
    for epoch in range(n_epochs):

        net.train()

        running_loss = 0.0
        for i, data in tqdm(enumerate(train_loader, 0)):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 2000 mini-batches
                print('TR [%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

        # Check validation set
        net.eval()

        with torch.no_grad():
            for i, data in tqdm(enumerate(val_loader, 0)):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data

                # forward + backward + optimize
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                # print statistics
                running_loss += loss.item()
                if i % 100 == 99:    # print every 2000 mini-batches
                    print('VAL [%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0

    print('Finished Training')

In [9]:
# net.load_state_dict(torch.load("./resnet18_mnist.pth"))
net.load_state_dict(torch.load("./resnet50_mnist.pth"))
# train()
# Save model
# torch.save(net.state_dict(), "./resnet18_mnist.pth")
# torch.save(net.state_dict(), "./resnet50_mnist.pth")

<All keys matched successfully>

In [10]:
with torch.no_grad():
    for i, data in tqdm(enumerate(val_loader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        print(labels)
        print(outputs)
        print(labels.size())
        print(outputs.size())
        
        _, predicted = torch.max(nn.Softmax()(outputs), 1)
        print(predicted)
        
        break
#         loss = criterion(outputs, labels.to(device))

        # print statistics


0it [00:00, ?it/s]<ipython-input-10-d2143b548c09>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  _, predicted = torch.max(nn.Softmax()(outputs), 1)
0it [00:00, ?it/s]

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1,
        7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5,
        1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1,
        0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0,
        3, 6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1,
        5, 9, 8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7])
tensor([[-3.9593, -0.8277, -2.5127,  ..., 13.1342, -3.4146, -0.5884],
        [-1.8355, -1.2160,  9.3036,  ..., -2.1379, -0.9067, -3.1126],
   